<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pyg/pointnet-classification/02_pointnet_plus_plus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{pyg-pointnet2-train} -->

# PointNet++ Model 



## Imports


In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

import random
from glob import glob           # ← re-added, in case you use it later
from tqdm.auto import tqdm
 
import wandb

import torch.nn.functional as F

import torch_geometric.transforms as T
from dataset import GainRegressionDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn.conv import PointConv
from torch_geometric.nn import MLP, fps, global_max_pool, radius
from model import PointNet2

2.5.1


## Initialize Weights & Biases

We need to call [`wandb.init()`](https://docs.wandb.ai/ref/python/init) once at the beginning of our program to initialize a new job. This creates a new run in W&B and launches a background process to sync data.

In [2]:
os.environ["WANDB_API_KEY"] = "5e8ac01bfe427d9d4c24a53896514c7eb3fc20e1"
# --- 1) WandB initialization --------------------------------------------
wandb_project   = "point_net_traj_planning"               #@param {"type": "string"}
wandb_run_name  = "final-experiment/gain-regression"  #@param {"type": "string"}

wandb.init(
    project=wandb_project,
    name=wandb_run_name,
    job_type="baseline-train"
)

# --- 2) Fill wandb.config -----------------------------------------------
config = wandb.config

# (a) Seed & reproducibility
config.seed = 4242  #@param {type:"number"}
random.seed(config.seed)
torch.manual_seed(config.seed)

# (b) Dataset parameters (replace ModelNet)
#     Point to the folder where your PLYs + labels.csv + opt_successfull.yaml live
config.data_root     = "/home/geriatronics/pmaf_ws/src/dataset_generator/data"  #@param {"type":"string"}
config.labels_csv    = "labels.csv"                                        #@param {"type":"string"}
config.success_yaml  = "opt_successfull.yaml"                         #@param {"type":"string"}

#     Number of points to sample from each cloud
config.npoints       = 2500  #@param {"type":"slider", min:256, max:4096, step:16}

# (c) Split / loader settings
config.test_size     = 0.20  #@param {"type":"slider", min:0.0, max:0.5, step:0.05}
config.batch_size    = 16    #@param {"type":"slider", min:4, max:128, step:4}
config.num_workers   = 0     #@param {"type":"slider", min:1, max:16, step:1}

# (d) Device
config.device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(config.device)

# (e) Model/hyperparameters (you can keep these or adjust)
config.set_abstraction_ratio_1 = 0.748   #@param {"type":"slider", min:0.1, max:1.0, step:0.01}
config.set_abstraction_radius_1 = 0.4817 #@param {"type":"slider", min:0.1, max:1.0, step:0.01}
config.set_abstraction_ratio_2 = 0.3316   #@param {"type":"slider", min:0.1, max:1.0, step:0.01}
config.set_abstraction_radius_2 = 0.2447 #@param {"type":"slider", min:0.1, max:1.0, step:0.01}
config.dropout                 = 0.1    #@param {"type":"slider", min:0.0, max:0.5, step:0.05}

# (f) Optimizer settings
config.learning_rate           = 1e-4  #@param {"type":"number"}
config.epochs                  = 10    #@param {"type":"slider", min:1, max:100, step:1}
config.num_visualization_samples = 20  #@param {"type":"slider", min:1, max:100, step:1}


wandb: Currently logged in as: mateus-salomao (mateus-salomao-technical-university-of-munich) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Load Dataset

We now load, preprocess and batch the dataset for training, validation/testing and visualization.

In [3]:
# Remove pre_transform and transform since GainRegressionDataset handles sampling & normalization internally

train_dataset = GainRegressionDataset(
    root         = config.data_root,
    labels_csv   = config.labels_csv,
    success_yaml = config.success_yaml,
    split        = "train",
    test_size     = config.test_size,
    random_state = config.seed,
    npoints      = config.npoints,
    augment      = True
)
val_dataset = GainRegressionDataset(
    root         = config.data_root,
    labels_csv   = config.labels_csv,
    success_yaml = config.success_yaml,
    split        = "val",
    test_size    = config.test_size,
    random_state = config.seed,
    npoints      = config.npoints,
    augment      = False
)

train_loader = DataLoader(
    train_dataset,
    batch_size   = config.batch_size,
    shuffle      = True,
    num_workers  = config.num_workers
)
val_loader = DataLoader(
    val_dataset,
    batch_size   = config.batch_size,
    shuffle      = False,
    num_workers  = config.num_workers
)

print(f"Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}")

Train samples: 224, Val samples: 57


## Implementing the PointNet++ Model using PyTorch Geometric

## Training PointNet++ and Logging Metrics on Weights & Biases

In [7]:
# Define PointNet++ model.
model = PointNet2(
    config.set_abstraction_ratio_1,
    config.set_abstraction_ratio_2,
    config.set_abstraction_radius_1,
    config.set_abstraction_radius_2,
    config.dropout
).to(device)

# Define Optimizer
optimizer = torch.optim.Adam(
    model.parameters(), lr=config.learning_rate
)

In [8]:

def train_step(
    epoch):
    """Training Step for Regression"""
    model.train()
    epoch_loss = 0.0
    num_batches = len(train_loader)

    progress_bar = tqdm(
        range(num_batches),
        desc=f"Training Epoch {epoch}/{config.epochs}"
    )
    data_iter = iter(train_loader)
    for _ in progress_bar:
        data = next(data_iter).to(device)
        # data.pos: [batch_size, npoints, 3]
        # data.y  : [batch_size, 36] (regression targets)

        optimizer.zero_grad()
        prediction = model(data)                   # [batch_size, 36]
        #print(f"Prediction shape: {prediction.shape}")
        #print(f"Label shape: {data.y.shape}")
        loss = F.mse_loss(prediction, data.y)       # Mean‐squared error
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / num_batches
    wandb.log({
        "Train/Loss": avg_loss,
        "Epoch": epoch
    })


def val_step(epoch):
    """Validation Step for Regression"""
    model.eval()
    epoch_loss = 0.0
    num_batches = len(val_loader)

    progress_bar = tqdm(
        range(num_batches),
        desc=f"Validation Epoch {epoch}/{config.epochs}"
    )
    data_iter = iter(val_loader)
    for _ in progress_bar:
        data = next(data_iter).to(device)
        with torch.no_grad():
            prediction = model(data)
            loss = F.mse_loss(prediction, data.y)
        epoch_loss += loss.item()

    avg_loss = epoch_loss / num_batches
    wandb.log({
        "Validation/Loss": avg_loss,
        "Epoch": epoch
    })




def save_checkpoint(epoch):
    """Save model + optimizer state as a W&B Artifact"""
    ckpt_path = f"checkpoint_epoch_{epoch}.pt"
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, ckpt_path)

    artifact_name = wandb.util.make_artifact_name_safe(
        f"{wandb.run.name}-{wandb.run.id}-checkpoint"
    )
    ckpt_artifact = wandb.Artifact(artifact_name, type="checkpoint")
    ckpt_artifact.add_file(ckpt_path)
    wandb.log_artifact(ckpt_artifact, aliases=["latest", f"epoch-{epoch}"])



In [ ]:
# Prepare an empty W&B Table for visualization
viz_table = wandb.Table(
    columns=[
        "Epoch",
        "PointCloud",
        "Prediction",
        "GroundTruth",
        "MSE_Error"
    ]
)

for epoch in range(1, config.epochs + 1):
    train_step(epoch)
    val_step(epoch)
   # visualize_evaluation(viz_table, epoch)
    save_checkpoint(epoch)

wandb.finish()

Training Epoch 1/10:   0%|          | 0/14 [00:00<?, ?it/s]

Validation Epoch 1/10:   0%|          | 0/4 [00:00<?, ?it/s]

Training Epoch 2/10:   0%|          | 0/14 [00:00<?, ?it/s]

Validation Epoch 2/10:   0%|          | 0/4 [00:00<?, ?it/s]

Training Epoch 3/10:   0%|          | 0/14 [00:00<?, ?it/s]

...

Training Epoch 9/10:   0%|          | 0/14 [00:00<?, ?it/s]

Validation Epoch 9/10:   0%|          | 0/4 [00:00<?, ?it/s]

Training Epoch 10/10:   0%|          | 0/14 [00:00<?, ?it/s]

Validation Epoch 10/10:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch,▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
Train/Loss,█▃
Validation/Loss,█▄▂
Epoch,10
Train/Loss,1.5246
Validation/Loss,1.35841


In [21]:
wandb.finish()

Next, you can check out the following notebook to learn how to run a hyperparameter sweep on our PointNet++ trainig loop using Weights & Biases:

|Tune Hyperparameters using Weights & Biases Sweep|[![](https://colab.research.google.com/assets/colab-badge.svg)](http://wandb.me/pyg-pointnet2-sweep)|